# SSW2022 Orbit Fit Challenge

# Orbit fitting of Gaia-like observations

In this hands-on activity you will learn to fit sky paths and orbits to absolute astrometry.  In the course of the activity you will:

1) load synthetic Gaia-like observations; 

2) fit a single-star model using least-squares, and assess the goodness of fit (whether a single-star model is an acceptable fit to the data);

3) add orbital motion to the single-star model and redo the fit and assess whether this is a good fit to the data;

4) visualize your fit with an orbital model and the synthetic Gaia observations superposed;

5) explore confidence regions on orbital parameters by computing the best $\chi^2$ as a function of period and eccentricity; and

6) assess what constraints you can place on the companion mass.

There may be a number of subtleties with the interpretations of the results, subtleties that are not easily expressed with something like ${\rm period}\,=\,x\pm \sigma_x\,{\rm days}$.  Keep this in mind as you think about how to interpret Gaia DR3 data, and the challenges that releasing a catalog like Gaia DR3 presents!

The first few cells will import necessary packages and load some data.

In [ ]:
# astropy imports
from astropy.table import QTable
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
plt.style.use('https://catcopy.ipac.caltech.edu/ssw/hands-on/ssw.mplstyle')

Two files, organized as .csv tables, are necessary for the analysis:

1) the file of synthetic Gaia observations;

2) the file of scanning law parameters.

The default unit of the Gaia along-scan position measurements is arcsec. Some of the units of time are provided in JD, but to better handle the treatment of the relevant parameters in the fits these will be transformed in Julian years. Angles are provided by default in degrees, but must be converted to radians for any calculation involving trigonometric functions.

The second file contains the list of observation times (the same as in the first file) and scanning law parameters.  You will need these to interpret the directions of the measurements given in the first file.  

In [ ]:
# Read data file with synthetic Gaia observations
v1298tau_data = QTable.read('https://catcopy.ipac.caltech.edu/ssw/hands-on/v1298tau_observations.csv')
# Read data file with scanning law parameters (angles and parallax factors)
gaia_scan_v1298tau = QTable.read('https://catcopy.ipac.caltech.edu/ssw/hands-on/gost_22.1.1_730490_2022-06-09-11-12-00_v1298_tau.csv')

In [ ]:
#fill time-series variables
juld = v1298tau_data['Time[JD]']
wobs = v1298tau_data['w[arcsec]']
sigw = v1298tau_data['err_w[arcsec]']

ref_ep = 2457389.0 # Reference Epoch (J2016.0)

#Fill scanning law parameter variables
theta = gaia_scan_v1298tau['scanAngle[rad]']
costheta = np.cos(theta)
sintheta = np.sin(theta)
parfac = gaia_scan_v1298tau['parallaxFactorAlongScan']

t = (juld - ref_ep)/365.25 # in Julian years from 2016.0

### The Best-Fit Single Star Model

To compute the best-fit sky path, we will be fitting a model to the one-dimensional position measurements.  The position of the star at any given time in $x$ (i.e. right ascension) will be 

$x = x_0 + v_x t + \varpi f(t)$

where $v_x$ is a velocity (the proper motion in right ascension, often denoted $\mu_{\alpha*}$, and $f(t)$ is the changing apparent position of the star due to parallax: the larger the parallax $\varpi$, the larger this motion.

The star actually moves in two dimensions, $x$ and $y$ (or right ascension $\alpha$ and declination $\delta$).  Gaia measures its position in just one dimension, at an angle $\theta$ away from the $y$ or $\delta$ axis (east of north).  So, the position that Gaia measures is 

$p_{\rm Gaia} = x \sin \theta + y \cos \theta$.

The parallax factor included above is the component of parallactic motion along the scan direction.  

So, the measured position by Gaia should be, for a single star (now using the standard variables $\alpha$ and $\delta$ instead of $x$ and $y$),

$p_{\rm Gaia} = (\alpha_0 + \mu_\alpha t)\sin \theta + (\delta_0 + \mu_\delta t)\cos \theta + \varpi \cdot {\it pf}$

where ${\it pf}$ is the *parallax factor*.  

One other important detail is the time $t$.  The baseline positions $\alpha_0$ and $\delta_0$ are the positions at $t=0$.  It makes more sense to measure time from a reference epoch during Gaia's observations (like taking $t=0$ at 2016.0) rather than taking $t=0$ at the year zero, extrapolating the model more than 2000 years into the past.  It actually makes it more challenging to solve numerically!  So, where I write $t$ above, I really mean the date as measured from the reference epoch of 2016.0.  This is already computed for you as the variable `t` (in units of Julian years).

**With all that said**, go ahead and fit the single-star model to the data.  Compute the best-fit $\chi^2$, with 

$\chi^2 = \sum_i (p_{\rm measured} - p_{\rm predicted})^2/\sigma_i^2$

where $\sigma_i$ is the measurement uncertainty for the $i$-th measurement (`sigw`) and $p_{\rm measured}$ is the measured position (`wobs`).  Compare your best-fit $\chi^2$ with the degrees of freedom, i.e., the number of position measurements (48) minus the number of fitted parameters (five).  Is the single-star model a good fit to the data?

### Now we will fit an orbit plus a five-parameter sky path.

This means that we need to compute an orbit!  This will involve the solution to the Kepler problem, which you can find in any celestial mechanics textbook.  We will repeat the necessary parts here so you don't have to go dig out a textbook.

The first step is to derive something called the *mean anomaly*.  This is just the orbital phase, reduced to the range $[0, 2\pi)$:

$M = (2\pi (t - t_{\rm peri})/T)~{\rm mod}~(2\pi)$

where mod means modulo (`%` in Python, but watch your parentheses!), $T$ is the period, and $t_{\rm peri}$ is the time of periastron.

From the mean anomaly, we compute the eccentric anomaly $E$ by solving Kepler's equation (which gives $E$ implicitly):

$M = E - e \sin E$

where $e$ is the eccentricity.  The simplest way to solve this is with Newton-Raphson iteration to find the root of $M - E + e \sin E$, using a good initial guess to help:

$E_0 = M + (e\sin M)/\left(\sqrt{1 - 2e\cos M + e^2}\right)$

$E_{n + 1} = E_n - f/f' = E_n - (M - E + e \sin E)/(e \cos E - 1)$

until convergence.  Once you have the eccentric anomaly (which requires three parameters: $e$, $T$, and $t_{\rm peri}$), you can use the Thiele-Innes equations for the position on the sky.  These are:

$X = \cos E - e$

$Y = (\sin E)\sqrt{1 - e^2}$

$\Delta \alpha = BX + GY$

$\Delta \delta = AX + FY$

In the equations above, $A$, $B$, $F$, and $G$ are constants, all in units of arcseconds, and $\Delta \alpha$ and $\Delta \delta$ represent the orbital motion in right ascension and declination.

**Perform a $\chi^2$ fit to a sky path with orbital motion.  What is your best-fit $\chi^2$, and how does it compare to the number of degrees of freedom (48 measurements minus 12 fitted parameters: 5 for the sky path and 7 for the orbit)?**

**A few helpful hints:**

Make sure that eccentricity $e$ stays between zero and one!  If you use `curve_fit`, the default is to give initial guesses of 1.0 for all parameters.  This is bad for eccentricity!

You might have to try a number of different starting guesses for $e$, $T$, and $t_{\rm peri}$ to find a good orbital fit.  

Once you have the eccentric anomaly (which requires $e$, $T$, and $t_{\rm peri}$), the orbital motion is linear in $B$, $G$, $A$, and $F$.  You can multiply the $\Delta \alpha$ and $\Delta \delta$ equations by $\sin \theta$ and $\cos \theta$, respectively, and then perform a linear fit for $\alpha_0$, $\delta_0$, $\mu_\alpha$, $\mu_\delta$, $\varpi$, $B$, $G$, $A$, and $F$.  This would be just like the fit you did earlier for the five astrometric parameters $\alpha_0$, $\delta_0$, $\mu_\alpha$, $\mu_\delta$, and $\varpi$.

### Optional: plotting

Plot your best-fit orbit together with the data.  To do this, you can plot your orbit using an array of times encompassing the full range of epochs observed by Gaia: plot $\Delta \alpha$ and $\Delta \delta$ using the Thiele-Innes constants.  

Plotting the observed points will probably be harder than plotting your best-fit synthetic orbit.  You can then plot the observed points in the following way:

Use your fitting routine to compute the residuals, ${\rm data} - {\rm model}$, for the observed positions `wobs`.  Then take the model positions (using only the orbital part, i.e., without the parallax and proper motion parts) and add these residuals to the model, so that you are doing something like ${\rm data} - {\rm model} + {\rm model}$.  This will get you the observations.  You will have to multiply the (one-dimensional) residuals times $\cos \theta$ to get the declination residuals and the residuals times $\sin \theta$ to get the right ascension residuals.

You can then plot the error bars by taking the measurement uncertainties `sigw` and connecting a line going from $(-\sigma_w \sin \theta, -\sigma_w \cos \theta)$ on one side to $(\sigma_w \sin \theta, \sigma_w \cos \theta)$ on the other side of each measured position.


## Peculiar note about plotting:
Find two best fit orbits by varying the starting guesses (e.g., try one with a period of 2 years and another with a period of 5 years). Plot both.
You should notice that the $\alpha$, $\delta$ "positions" of the observations (the points with 1d error bars) actually change depending on what the best fit orbit is.

Is something wrong?

Answer: This is bizarre, but nothing is wrong. This is just the peculiar behavior of fitting to single dimensional observations. Gaia only observes the projection of the star's 2d position onto a 1d axis tilted at an angle $\theta$ (the scan angle). So in a sense, all you have is a line along which the star must have existed (or a broad swath, because there is uncertainty). The line makes an angle $\theta + 90^{\circ}$ East of North. 

### Now we will explore confidence intervals on our parameters.

Rather than computing a covariance matrix, please take the following approach.  Hold one parameter, either eccentricity or period, fixed, and optimize all of the other parameters (all but one).  Do this at a bunch of eccentricities, so that you can make a plot of $\chi^2$ as a function of eccentricity from $e=0.3$ to $e=0.95$.  Use this plot to estimate a $1\sigma$ confidence interval ($\chi^2 < \chi^2_{\rm min} + 1$ if everything is Gaussian).  

Next, make a plot of $\chi^2$ as a function of period from a period of 2 years to a period of 20 years.  What confidence interval would you place on the period?  

If you have the time and motivation, also make a plot of the best-fit eccentricity as a function of period.  You might notice that there are some well-fitting orbits at very high eccentricity.  Is there any reason to distrust such orbits?

Think about how your results above might, or might not, be reflected in Gaia's data release.

### Finally, derive the mass of the companion.

To derive the mass, you'll need to do some arithmetic with the Thiele-Innes constants within the optimization.  We define these as

$A = {\rm sma} \cdot (\cos \Omega \cos \omega - \sin \Omega \sin \omega \cos i)$

$B = {\rm sma} \cdot (\sin \Omega \cos \omega + \cos \Omega \sin \omega \cos i)$

$F = {\rm sma} \cdot (-\cos \Omega \sin \omega - \sin \Omega \cos \omega \cos i)$

$G = {\rm sma} \cdot (-\sin \Omega \sin \omega + \cos \Omega \cos \omega \cos i)$

where sma is the semimajor axis in units of arcseconds (you can look up the details in a celestial mechanics book).  With the helper variables

$u = \frac{1}{2} (A^2 + B^2 + F^2 + G^2)$

$v = BF - AG$

you can get the semimajor axis (in units of arcseconds) by

${\rm sma} = \sqrt{u + \sqrt{u^2 - v^2}}$

To convert to AU you can divide by the parallax, which is one of the parameters you are deriving.  You can then use this semimajor axis, which refers to the orbit of the primary about the barycenter (a fraction $M_{\rm sec}/M_{\rm tot}$ of the total semimajor axis), to derive a companion mass.  You'll need the mass of the primary star; you can take it to be 1.17$\,M_\odot$.  You might also find Kepler's Third Law useful: $T^2 = a^3/M_{\rm tot}$, with $T$ in years, $a$ in AU, and $M$ in Solar masses.

Using your results here, together with what you found for period and eccentricity, what sort of constraints can you place on the companion mass?